# **VINTAGE ANALYSIS**

In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import utils.utils as utils

# !pip freeze > ../requirements.txt

In [150]:
data = pd.read_csv(f'..\data\data_vintage.csv')

In [151]:
# Ordering the STATUS by DPD:
status_order = ['C', 'X', 'DPD0', 'DPD1-30', 'DPD31-60', 'DPD61-90', 'DPD91-120', 'DPD120+']
data['STATUS'] = pd.Categorical(data['STATUS'], categories=status_order, ordered=True)
data = data.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=True)

In [152]:
def _flag_bad(data, threshold='DPD61-90'):
    threshold = threshold
    mask = data[data['STATUS'] >= threshold]
    first_mask = mask.loc[mask.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmin()]
    data = data.merge(first_mask, on='SK_ID_BUREAU', suffixes=('', '_FIRST'))
    data['FLAG_BAD'] = (data['MONTHS_BALANCE'] >= data['MONTHS_BALANCE_FIRST']).astype(int)
    data = data.drop(['MONTHS_BALANCE_FIRST', 'STATUS_FIRST'], axis = 1)
    return(data)
data = _flag_bad(data)

In [157]:
data = data[data['MONTHS_BALANCE'] >= 24]

In [158]:
data

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,FLAG_BAD
23,5001797,24,C,1
24,5001797,25,C,1
25,5001797,26,C,1
26,5001797,27,C,1
27,5001797,28,C,1
...,...,...,...,...
253003,6842885,56,DPD120+,1
253004,6842885,57,DPD120+,1
253005,6842885,58,DPD120+,1
253006,6842885,59,DPD120+,1


In [110]:
pivot = data.pivot_table(index='MONTHS_BALANCE', columns='MOB', values='FLAG_BAD', aggfunc='sum')

In [111]:
pivot

MOB,1,2,3,4,5,6,7,8,9,10,...,28,29,30,31,32,33,34,35,36,37
MONTHS_BALANCE,,,,,,,,,,,,,,,,,,,,,
24,3053.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,3127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,3198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,3259.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,3326.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,3375.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,3424.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3481.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3529.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
